In [1]:
!pip install transformers

# summarization using auto-tokenizer

In [2]:
from bs4 import BeautifulSoup
import requests

In [50]:
website_url = "https://www.cbr.com/batman-dark-knight-trilogy-life-lessons/"

In [51]:
r = requests.get(website_url);

In [52]:
short = BeautifulSoup(r.text,"html.parser")
results = short.find_all(['h1','h2','p']);

In [53]:
results

[<h1 class="hide" itemprop="headline">www.cbr.com</h1>,
 <h1 class="article-title">Batman: 10 Life Lessons From Nolan's Dark Knight Trilogy</h1>,
 <p class="article-excerpt">The Dark Knight trilogy tells the story of Batman &amp; his superhero life, teaching important lessons along the way.</p>,
 <p>It's now been over 15 years since Christopher Nolan made his name a mainstay in the world of cinema with the first installment of his incredible trilogy, <a href="https://www.cbr.com/tag/the-dark-knight-trilogy/"><em>The Dark Knight</em></a>, and it's fair to say that they still are some of the best movies of all time, not even just in the superhero genre. But, what makes them so incredible is still up for debate.</p>,
 <p><span class="related-single">RELATED: <a href="https://www.cbr.com/trivia-factst-make-no-sense-dark-knight-trilogy/">10 Things That Make Zero Sense About The Dark Knight Trilogy</a></span></p>,
 <p>Of course, the movie is spearheaded by one of the most accomplished and sk

In [54]:
blocked = ['embed','search','sharer','intent','share','watch','subscribe','sitemap','contact us','privacy policy','Did you find this story helpful?','sign up','login','copyright','terms and conditions','call us','all rights reserved']
ext_text = [result.text for result in results]

In [55]:
texts=''
for i in range(len(ext_text)):
    for block in blocked:
        if block in ext_text[i].lower():
            break
    else:
        texts+=ext_text[i]
    

In [56]:
texts

'www.cbr.comBatman: 10 Life Lessons From Nolan\'s Dark Knight TrilogyThe Dark Knight trilogy tells the story of Batman & his superhero life, teaching important lessons along the way.It\'s now been over 15 years since Christopher Nolan made his name a mainstay in the world of cinema with the first installment of his incredible trilogy, The Dark Knight, and it\'s fair to say that they still are some of the best movies of all time, not even just in the superhero genre. But, what makes them so incredible is still up for debate.RELATED:\xa010 Things That Make Zero Sense About The Dark Knight TrilogyOf course, the movie is spearheaded by one of the most accomplished and skilled directors ever in Christopher Nolan. It\'s also filled with a star-studded cast with actors like Christain Bale, Michael Caine, and Heath Ledger all featuring.\xa0And despite its great story structure and compelling characters, arguably the most important reason why fans love the trilogy so much is the powerful messag

In [36]:
len(texts)

4616

In [10]:
from transformers import AutoTokenizer,AutoModelWithLMHead

In [11]:
tokenizer=AutoTokenizer.from_pretrained('t5-base')
model=AutoModelWithLMHead.from_pretrained('t5-base',return_dict=True)

C:\Users\TEST01\anaconda3\envs\deeplearning\lib\site-packages\transformers\models\auto\modeling_auto.py:806: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [74]:
inputs=tokenizer.encode("summarize: " + texts,return_tensors='pt',max_length=512,truncation=True)

In [75]:
outputs = model.generate(inputs, max_length=300, min_length=80, length_penalty=5, num_beams=2)

In [76]:
summary = tokenizer.decode(outputs[0])

In [77]:
summary

'<pad> the dark knight trilogy tells the story of Batman & his superhero life, teaching important lessons along the way. what makes them so incredible is still up for debate. what makes them so incredible is the powerful messages and lessons to be learned throughout. the typical superhero movie deals with two main types of characters: the hero, who will stop at nothing to protect everyone in the world.</s>'

# summarization using pipeline

In [ ]:
!pip install transformers
from transformers import pipeline

In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [ ]:
summarizer = pipeline('summarization')

In [ ]:
r = requests.get(website_url);

In [ ]:
short = BeautifulSoup(r.text,"html.parser")
results = short.find_all(['h1','h2','p']);

In [ ]:
results

In [ ]:
blocked = ['embed','search','sharer','intent','share','watch','subscribe','sitemap','contact us','privacy policy','Did you find this story helpful?','sign up','login','copyright','terms and conditions','call us','all rights reserved']
ext_text = [result.text for result in results]

In [ ]:
final_article = ' '.join(ext_text);

In [ ]:
final_article

In [ ]:
final_article = final_article.replace('.','.<eos>')
final_article = final_article.replace('!','!<eos>')
final_article = final_article.replace('?','?<eos>')
final_sentences = final_article.split('<eos>')

In [ ]:
total_words = 500
current_words = 0
words = []
for sentence in final_sentences:
    if(len(words)==current_words+1):
        if(len(words[current_words])) + len(sentence.split(' ')) <= total_words:
            words[current_words].extend(sentence.split(' '))
        else:
            current_words +=1;
            words.append(sentence.split(' '))
    else:
        print(current_words)
        words.append(sentence.split(' '))

In [ ]:
for word_id in range(len(words)):
    words[word_id] = ' '.join(words[word_id])

In [ ]:
final_summary = summarizer(words, max_length=300, min_length=50, do_sample=False)

In [ ]:
finalsummary=' '.join([summary['summary_text'] for summary in final_summary])